In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation parameters
num_days = 7
hours_per_day = 24
total_hours = num_days * hours_per_day
time_index = pd.date_range(start='2023-01-01', periods=total_hours, freq='H')

num_cows = 10
field_size = (100, 100)  # Field dimensions in meters

def generate_heat_status(total_hours):
    heat_duration = np.random.randint(8, 13)
    heat_start = np.random.randint(0, total_hours - heat_duration)
    is_in_heat = pd.Series(np.zeros(total_hours, dtype=bool), index=time_index)
    is_in_heat.iloc[heat_start:heat_start + heat_duration] = True
    return is_in_heat

class Cow:
    def __init__(self, cow_id, is_in_heat, field_size):
        self.cow_id = cow_id
        self.is_in_heat = is_in_heat
        self.field_size = field_size
        self.position = np.array([
            np.random.uniform(0, field_size[0]),
            np.random.uniform(0, field_size[1])
        ])
        self.positions = [self.position.copy()]
        self.activities = []
    
    def move(self, current_time, t):
        activity = self.determine_activity(current_time)
        self.activities.append(activity)
        
        if activity == 'Sleeping':
            step = np.array([0, 0])
        elif activity == 'Grazing':
            step = np.random.normal(loc=0, scale=0.2, size=2)
        elif activity == 'Walking':
            step = np.random.normal(loc=0, scale=0.5, size=2)
        elif activity == 'Mounting':
            step = np.random.normal(loc=0, scale=1.0, size=2)
        else:
            step = np.random.normal(loc=0, scale=0.1, size=2)
        
        self.position += step
        self.position[0] = np.clip(self.position[0], 0, self.field_size[0])
        self.position[1] = np.clip(self.position[1], 0, self.field_size[1])
        self.positions.append(self.position.copy())
    
    def determine_activity(self, current_time):
        hour = current_time.hour
        if 22 <= hour or hour < 5:
            return 'Sleeping'
        elif 5 <= hour < 7:
            return 'Walking'
        elif 7 <= hour < 19:
            if self.is_in_heat[current_time]:
                return np.random.choice(['Grazing', 'Walking', 'Mounting'], p=[0.3, 0.4, 0.3])
            else:
                return np.random.choice(['Grazing', 'Walking'], p=[0.7, 0.3])
        elif 19 <= hour < 22:
            return 'Grazing'
        else:
            return 'Resting'

def calculate_movement_speed(positions):
    positions = np.array(positions)
    diffs = np.diff(positions, axis=0)
    speeds = np.linalg.norm(diffs, axis=1)
    speeds = np.insert(speeds, 0, 0)
    return speeds

all_cows_data = []

for cow_id in range(1, num_cows + 1):
    is_in_heat = generate_heat_status(total_hours)
    cow = Cow(cow_id, is_in_heat, field_size)
    
    for t in range(total_hours):
        current_time = time_index[t]
        cow.move(current_time, t)
    
    positions = cow.positions[:-1]  # Exclude the last extra position
    speeds = calculate_movement_speed(positions)
    
    cow_data = pd.DataFrame({
        'CowID': cow_id,
        'Timestamp': time_index,
        'X_Position': [pos[0] for pos in positions],
        'Y_Position': [pos[1] for pos in positions],
        'Activity': cow.activities,
        'InHeat': is_in_heat.values,
        'Speed': speeds
    })
    all_cows_data.append(cow_data)

all_cows_data = pd.concat(all_cows_data)
all_cows_data.set_index(['CowID', 'Timestamp'], inplace=True)

# Convert positions to GPS coordinates
ref_latitude = 40.0
ref_longitude = -80.0

degrees_per_meter_latitude = 1 / 111_000
degrees_per_meter_longitude = 1 / (111_000 * np.cos(np.deg2rad(ref_latitude)))

all_cows_data['Latitude'] = ref_latitude + all_cows_data['Y_Position'] * degrees_per_meter_latitude
all_cows_data['Longitude'] = ref_longitude + all_cows_data['X_Position'] * degrees_per_meter_longitude


/tmp/ipykernel_4179850/1937308365.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start='2023-01-01', periods=total_hours, freq='H')


In [4]:
!pip install folium

In [5]:
import folium

# Create a map centered at the field's center
field_center = [
    ref_latitude + (field_size[1] / 2) * degrees_per_meter_latitude,
    ref_longitude + (field_size[0] / 2) * degrees_per_meter_longitude
]
m = folium.Map(location=field_center, zoom_start=18)

# Plot cows' positions at a specific time
time_to_plot = '2023-01-03 12:00:00'

for cow_id in range(1, num_cows + 1):
    cow_data = all_cows_data.xs((cow_id, time_to_plot))
    folium.CircleMarker(
        location=[cow_data['Latitude'], cow_data['Longitude']],
        radius=5,
        color='red' if cow_data['InHeat'] else 'blue',
        popup=f'Cow {cow_id} - {cow_data["Activity"]}',
        fill=True
    ).add_to(m)

# Display the map
m


In [6]:
all_cows_data

X_Position  Y_Position  Activity  InHeat     Speed  \
CowID Timestamp                                                                 
1     2023-01-01 00:00:00   58.667203   89.776251  Sleeping   False  0.000000   
      2023-01-01 01:00:00   58.667203   89.776251  Sleeping   False  0.000000   
      2023-01-01 02:00:00   58.667203   89.776251  Sleeping   False  0.000000   
      2023-01-01 03:00:00   58.667203   89.776251  Sleeping   False  0.000000   
      2023-01-01 04:00:00   58.667203   89.776251  Sleeping   False  0.000000   
...                               ...         ...       ...     ...       ...   
10    2023-01-07 19:00:00   45.834018  100.000000   Grazing   False  0.233425   
      2023-01-07 20:00:00   45.702733  100.000000   Grazing   False  0.131285   
      2023-01-07 21:00:00   46.004982   99.872332   Grazing   False  0.328106   
      2023-01-07 22:00:00   46.083770   99.533661  Sleeping   False  0.347715   
      2023-01-07 23:00:00   46.083770   99.533661  Sleeping   False  0.000000   

                            Latitude  Longitude  
CowID Timestamp                                  
1     2023-01-01 00:00:00  40.000809 -79.999310  
      2023-01-01 01:00:00  40.000809 -79.999310  
      2023-01-01 02:00:00  40.000809 -79.999310  
      2023-01-01 03:00:00  40.000809 -79.999310  
      2023-01-01 04:00:00  40.000809 -79.999310  
...                              ...        ...  
10    2023-01-07 19:00:00  40.000901 -79.999461  
      2023-01-07 20:00:00  40.000901 -79.999463  
      2023-01-07 21:00:00  40.000900 -79.999459  
      2023-01-07 22:00:00  40.000897 -79.999458  
      2023-01-07 23:00:00  40.000897 -79.999458  

[1680 rows x 7 columns]